In [1]:
def UWPGMA(frame, weighted=True):
    
    import pandas as pd
    import numpy as np
    
    def min_ind(frame):
        ind=frame.values.argmin()
        row=ind//frame.shape[0]
        col=ind%frame.shape[1]
        name_t=(frame.index[row], frame.columns[col])
        return name_t
    
    Weights={key: 1 for key in frame.index}
    
    def weight(el):
        return 1 if weighted else Weights[el]
        
    def weights_upd(el_1, el_2):
        Weights[el_1+el_2]=Weights[el_1]+Weights[el_2]
        
    def format_matrix(frame, el_1, el_2):
        temp=frame.iloc[(frame.index!=el_1) & (frame.index!=el_2), (frame.columns!=el_1) & (frame.columns!=el_2)]
        distances=[]
        
        for c in temp.index:
            f_1=(frame.loc[c, el_1] if frame.loc[c, el_1]!=np.inf else frame.loc[el_1, c])
            f_2=(frame.loc[c, el_2] if frame.loc[c, el_2]!=np.inf else frame.loc[el_2, c])
            dist=(weight(m_name[0])*f_1+weight(m_name[1])*f_2)/(weight(m_name[0])+weight(m_name[1]))
            distances.append(dist)
        
        temp[el_1+el_2]=distances
        row_to_app=pd.Series([np.inf]*temp.shape[1], index=temp.columns, name=el_1+el_2)
        temp=temp.append(row_to_app)
        
        return temp
        
    Tree_struct={}
    Inner_dist={key: 0 for key in frame.index}
    
    def merge(el_1, el_2):
        Dist=(frame.loc[el_1, el_2] if frame.loc[el_1, el_2]!=np.inf else frame.loc[el_2, el_1])
        m_1=(Tree_struct[el_1] if el_1 in Tree_struct else el_1)
        m_2=(Tree_struct[el_2] if el_2 in Tree_struct else el_2)
        Tree_struct[el_1+el_2]='({}: {}, {}: {})'.format(m_1, Dist/2-Inner_dist[el_1], 
                                                         m_2, Dist/2-Inner_dist[el_2])
        Inner_dist[el_1+el_2]=Dist/2
            
    while np.sum(frame.values.flatten()!=np.inf)!=1:
        
        m_name=min_ind(frame)
        
        merge(*m_name)
        weights_upd(*m_name)
        frame=format_matrix(frame, *m_name)
        
    final_name=min_ind(frame)
    merge(*final_name)
        
    return frame, Tree_struct[final_name[0]+final_name[1]]

In [143]:
def NeighborJoin(frame):
    
    import pandas as pd
    import numpy as np
    
    '''
    def corr_ind(ind):
        return '-'.join(sorted(index.split('-')))
    '''

    def swap_if(*args):
        i=frame.index.get_loc(args[0])
        j=frame.columns.get_loc(args[1])
        return args[::-1] if i>j else args
        
    Terms=pd.Series([np.sum([frame.loc[swap_if(col, comp)] if comp!=col else 0 for comp in frame.columns])\
                          for col in frame.columns], 
                          name='Common_Terms', index=frame.columns)
    
    #Очень важно, TERMS нужно обновлять для всех последовательностей, когда какая-либо удаляется, то есть
    #становится узлом, и добавляется новый узел!
    #Inner_dist={}#Надо ли??
    
    def delete_Terms(Terms, nodes):
        Terms=Terms.drop(nodes)
        for i in Terms.index:
            Terms[i]=Terms[i]-frame.loc[swap_if(i, nodes[0])]-frame.loc[swap_if(i, nodes[1])]
        return Terms
    
    def add_Terms(Terms, new_node):
        for i in Terms.index:
            Terms[i]=Terms[i]+frame.loc[swap_if(i, new_node)]
        Terms[new_node]=np.sum([frame.loc[swap_if(new_node, comp)] if comp!=new_node else 0\
                                for comp in frame.columns])
        return Terms
    
    Tree_struct={}
    
    def merge(dist, el_1, el_2):
        m_1=(Tree_struct[el_1] if el_1 in Tree_struct else el_1)
        m_2=(Tree_struct[el_2] if el_2 in Tree_struct else el_2)
        D_1=0.5*(dist+2*(Terms[el_1]-frame.loc[swap_if(el_1, el_2)])/(frame.shape[0]-2))
        D_2=0.5*(dist+2*(Terms[el_2]-frame.loc[swap_if(el_1, el_2)])/(frame.shape[0]-2))
        Tree_struct[el_1+el_2]='({}: {}, {}: {})'.format(m_1, D_1, 
                                                         m_2, D_2)
    
    def update_frame(frame, dist, el_1, el_2):
        temp=frame.iloc[(frame.index!=el_1) & (frame.index!=el_2), \
                        (frame.columns!=el_1) & (frame.columns!=el_2)]
        distances=[]
        
        for c in temp.index:
            D=0.5*(frame.loc[swap_if(c, el_1)]+frame.loc[swap_if(c, el_2)]-dist)
            distances.append(D)
        
        temp[el_1+el_2]=distances
        row_to_app=pd.Series([np.inf]*temp.shape[1], index=temp.columns, name=el_1+el_2)
        temp=temp.append(row_to_app)
        return temp
    
    def closest_pair(frame):
        row=''
        col=''
        minimum=np.inf
        for j in range(1, frame.shape[1]):
            for i in range(j):
                #print(frame.index[i], frame.index[j])
                #print((Terms[frame.index[i]]-frame.iloc[i, j])/(frame.shape[0]-2))
                #print((Terms[frame.index[j]]-frame.iloc[i, j])/(frame.shape[0]-2))
                Dist=frame.iloc[i, j]\
                -(Terms[frame.index[i]]-frame.iloc[i, j])/(frame.shape[0]-2)\
                -(Terms[frame.index[j]]-frame.iloc[i, j])/(frame.shape[0]-2)
                #print(Dist)
                if Dist<minimum:
                    minimum=Dist
                    row=frame.index[i]
                    col=frame.index[j]
        return [row, col], minimum
    
    while np.sum(frame.values.flatten()!=np.inf)!=1:
        
        m_name, dist=closest_pair(frame)
        merge(dist, *m_name)
        Terms=delete_Terms(Terms, m_name)
        frame=update_frame(frame, dist, *m_name)
        Terms=add_Terms(Terms, m_name[0]+m_name[1])
        
    final_name=[frame.index[0], frame.columns[-1]]
    dist=frame.iloc[0, -1]
    Tree_struct[''.join(final_name)]='({}: {}, {}: {})'.format(Tree_struct[final_name[0]], dist/2, 
                                                        Tree_struct[final_name[1]], dist/2)
    return frame, Tree_struct[''.join(final_name)]

In [5]:
import pandas as pd
import numpy as np

# Testing U(W)PGMA

In [142]:
Test_1=pd.DataFrame([[np.inf, 16, 16, 10],[np.inf, np.inf, 8, 8], 
                     [np.inf, np.inf, np.inf, 4], [np.inf, np.inf, np.inf, np.inf]],
                   index=['A', 'B', 'C', 'D'], columns=['A', 'B', 'C', 'D'])
Test_1

,A,B,C,D
A,inf,16.000000,16.000000,10.000000
B,inf,inf,8.000000,8.000000
C,inf,inf,inf,4.000000
D,inf,inf,inf,inf


In [4]:
F, T_s=UWPGMA(Test_1)
F

,A,BCD
A,inf,14.500000
BCD,inf,inf


In [5]:
T_s

'(A: 7.25, (B: 4.0, (C: 2.0, D: 2.0): 2.0): 3.25)'

In [6]:
F, T_s=UWPGMA(Test_1, weighted=False)
F

,A,BCD
A,inf,14.000000
BCD,inf,inf


In [7]:
T_s

'(A: 7.0, (B: 4.0, (C: 2.0, D: 2.0): 2.0): 3.0)'

In [6]:
Test_2=pd.DataFrame([[np.inf, 5, 4, 7, 6, 8],[np.inf, np.inf, 7, 10, 9, 11], 
                     [np.inf, np.inf, np.inf, 7, 6, 8], [np.inf, np.inf, np.inf, np.inf, 5, 9],
                     [np.inf, np.inf, np.inf, np.inf, np.inf, 8],
                     [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]],
                   index=['A', 'B', 'C', 'D', 'E', 'F'], columns=['A', 'B', 'C', 'D', 'E', 'F'])
Test_2

,A,B,C,D,E,F
A,inf,5.000000,4.000000,7.000000,6.000000,8.000000
B,inf,inf,7.000000,10.000000,9.000000,11.000000
C,inf,inf,inf,7.000000,6.000000,8.000000
D,inf,inf,inf,inf,5.000000,9.000000
E,inf,inf,inf,inf,inf,8.000000
F,inf,inf,inf,inf,inf,inf


In [9]:
F, T_s=UWPGMA(Test_2)
F

,F,DEBAC
F,inf,9.000000
DEBAC,inf,inf


In [10]:
T_s

'(F: 4.5, ((D: 2.5, E: 2.5): 1.5, (B: 3.0, (A: 2.0, C: 2.0): 1.0): 1.0): 0.5)'

In [11]:
F, T_s=UWPGMA(Test_2, weighted=False)
F

,F,DEBAC
F,inf,8.800000
DEBAC,inf,inf


In [12]:
T_s

'(F: 4.4, ((D: 2.5, E: 2.5): 1.25, (B: 3.0, (A: 2.0, C: 2.0): 1.0): 0.75): 0.6500000000000004)'

# Testing Neighbor-Join Algorithm

In [144]:
F, T_s=NeighborJoin(Test_1)

In [145]:
F

,CD,AB
CD,inf,30.000000
AB,inf,inf


In [147]:
T_s

'((C: 3.5, D: 0.5): 15.0, (A: 10.5, B: 5.5): 15.0)'

In [148]:
F, T_s=NeighborJoin(Test_2)

In [149]:
F

,CF,ABDE
CF,inf,36.250000
ABDE,inf,inf


In [150]:
T_s

'((C: 2.5, F: 5.5): 18.125, ((A: 1.0, B: 4.0): 9.249999999999996, (D: 3.0, E: 2.0): 10.916666666666668): 18.125)'